In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### Experiment 1: Train Logistic Regression Classifier

In [4]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



### Experiment 2: Train Random Forest Classifier

In [5]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.63      0.76        30

    accuracy                           0.96       300
   macro avg       0.96      0.81      0.87       300
weighted avg       0.96      0.96      0.96       300



### Experiment 3: Train XGBoost

In [6]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



### Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [7]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

In [8]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



<h2 align="center" style="color:blue">Track Experiments Using MLFlow</h2>

In [9]:
models = [
    (
        "Logistic Regression", 
        LogisticRegression(C=1, solver='liblinear'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        RandomForestClassifier(n_estimators=30, max_depth=3), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [10]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [11]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [12]:
reports[0]

{'0': {'precision': 0.9454545454545454,
  'recall': 0.9629629629629629,
  'f1-score': 0.9541284403669725,
  'support': 270.0},
 '1': {'precision': 0.6,
  'recall': 0.5,
  'f1-score': 0.5454545454545454,
  'support': 30.0},
 'accuracy': 0.9166666666666666,
 'macro avg': {'precision': 0.7727272727272727,
  'recall': 0.7314814814814814,
  'f1-score': 0.749791492910759,
  'support': 300.0},
 'weighted avg': {'precision': 0.9109090909090909,
  'recall': 0.9166666666666666,
  'f1-score': 0.91326105087573,
  'support': 300.0}}

In [13]:
reports[0]['0']

{'precision': 0.9454545454545454,
 'recall': 0.9629629629629629,
 'f1-score': 0.9541284403669725,
 'support': 270.0}

In [14]:
reports[0]['1']

{'precision': 0.6,
 'recall': 0.5,
 'f1-score': 0.5454545454545454,
 'support': 30.0}

In [15]:
reports[0]['macro avg']

{'precision': 0.7727272727272727,
 'recall': 0.7314814814814814,
 'f1-score': 0.749791492910759,
 'support': 300.0}

In [16]:
reports[0]['weighted avg']

{'precision': 0.9109090909090909,
 'recall': 0.9166666666666666,
 'f1-score': 0.91326105087573,
 'support': 300.0}

In [17]:
reports[0]['accuracy']

0.9166666666666666

In [19]:
for idx, element in enumerate(models):
    print(element)

('Logistic Regression', LogisticRegression(C=1, solver='liblinear'), (array([[ 1.18673836,  1.51144074,  0.78490373, ..., -0.61229492,
        -0.13830257, -0.24753395],
       [-1.28810271, -1.03855344, -2.07092052, ...,  0.49607021,
        -1.50376955,  0.62474155],
       [ 1.66393774,  1.55142135,  2.25024183, ..., -0.69955586,
         1.36600648, -0.68290518],
       ...,
       [ 0.43101615,  0.90013637, -0.42606094, ..., -0.32069604,
        -1.01508454,  0.11782042],
       [ 0.79839935,  1.5003473 , -0.45098948, ..., -0.54728572,
        -1.42140103,  0.11944858],
       [ 0.67367695,  1.27538516, -0.39960348, ..., -0.464427  ,
        -1.22522394,  0.10635794]], shape=(700, 10)), array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       

In [24]:
for idx, element in enumerate(models):
    model_name= element[0]
    # print(model_name)
    if "XGB" in model_name:
        print(f'yes found XG model and here is model name---------- {model_name}')

yes found XG model and here is model name---------- XGBClassifier
yes found XG model and here is model name---------- XGBClassifier With SMOTE


In [26]:

mlflow.set_experiment("Anomaly Detection_from_codebasics")
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_param("model", model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_class_0', report['0']['f1-score'])
        mlflow.log_metric('f1_scorel_class_1', report['1']['f1-score'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])        
        mlflow.log_metric('recall_score_macro', report['macro avg']['recall'])   

        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2026/01/22 23:42:31 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection_from_codebasics' does not exist. Creating a new experiment.
2026/01/22 23:42:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/2/runs/478df234f29d470b95094796c88bcb37
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


2026/01/22 23:43:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/2/runs/33ebc41027f841259aff729397d63917
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


2026/01/22 23:43:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/2/runs/d60158d596524ec1aec227a9b34be5c0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


2026/01/22 23:43:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/2/runs/5df235c8fe0b423db8aa1ab7c61e384f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
